# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [33]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets

In [34]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_PUBLIC_TOKEN")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [35]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [54]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
sfo_units=sfo_data.groupby(["year","housing_units"]).mean().drop(columns="sale_price_sqr_foot").drop(columns="gross_rent")

avg_units=sfo_units.hvplot.line(x="year", xlabel="year", y="housing_units", ylabel="Housing Units", yformatter="%.0f", title="Average Housing Units Per Year")

In [37]:
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
avg_rent =sfo_data.groupby(["year","gross_rent"]).mean().drop(columns="housing_units")
avg_rent_plot=avg_rent.hvplot.line(x="year", xlabel= "Year", y="gross_rent", ylabel = "Gross Rent", title="Average Rent Per Month in San Francisco")

In [55]:
def average_sales_price():
    """Average Sales Price Per Year."""
avg_sales_plot=avg_rent.hvplot.line(x="year", xlabel = "Year", y="sale_price_sqr_foot", ylabel = "Average Sale Price Per Square Foot", title="Average Sale Price Per Square Foot in San Francisco")

In [39]:
sfo_neighborhood=sfo_data.groupby(["year","neighborhood"]).mean().reset_index()
sfo_neighborhood_sale=sfo_neighborhood
#.drop(columns="gross_rent").drop(columns="housing_units")
sfo_neighborhood_sale



list_of_neighborhoods = sfo_neighborhood_sale['neighborhood'].values.tolist()


def plot_price_per_square_foot(neighborhood):

    
    return sfo_neighborhood_sale[sfo_neighborhood_sale["neighborhood"]==neighborhood].hvplot.line(
        x="year",
        xlabel="Year",
        y="sale_price_sqr_foot",
        ylabel="Sale Price per Square Foot",
        title="San Francisco Price Per Square Foot by Neighborhood",
        groupby="neighborhood",
        
        
    )


# Render plot with Panel interactive widget
interact(plot_price_per_square_foot, neighborhood=list_of_neighborhoods)

Column
    [0] Column
        [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
    [1] Row
        [0] Row(name='interactive02624')
            [0] HoloViews(DynamicMap, name='interactive02624')
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square'], value='Alamo Square', width=250)
                [1] VSpacer()

In [67]:
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
sfo_neighborhood_exp=sfo_neighborhood_sale.groupby("neighborhood").mean().drop(columns="year").reset_index()


sfo_top_10=sfo_neighborhood_exp.sort_values("sale_price_sqr_foot", ascending=False).set_index("neighborhood").iloc[0:9]
sfo_top_10.reset_index(inplace=True)
sfo_top_10.head()
exp_neighborhoods= sfo_top_10.hvplot.bar(x="neighborhood", xlabel = "Neighborhood", y="sale_price_sqr_foot", ylabel = "Sale Price Per Square Foot", rot =90, title="Sale Price per Square Foot in Most Expensive San Francisco Neighborhoods")

In [41]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
par_coor=px.parallel_coordinates(sfo_top_10, color="sale_price_sqr_foot")

In [42]:
def parallel_categories():
    """Parallel Categories Plot."""
    
par_categories=px.parallel_categories(
    sfo_top_10.round(2),
    dimensions=["neighborhood", "housing_units", "gross_rent", "sale_price_sqr_foot"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neighborhood": "Neighborhood",
        "housing_units": "Number of Housing Units",
        "gross_rent": "Rent",
        "sale_price_sqr_foot":"Sale Price per Square Foot"
    },
)

In [60]:
def neighborhood_map():
    """Neighborhood Map"""
df_neighborhood_locations.rename(columns={"Neighborhood":"neighborhood"}, inplace=True)
df_neighborhood_locations.head()
    
neighborhood_map=pd.merge(sfo_neighborhood_exp, df_neighborhood_locations, how="inner", on="neighborhood")
neighborhood_map

map_1 = px.scatter_mapbox(
    neighborhood_map,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    zoom=10,
    color="neighborhood")

map_2 = px.scatter_mapbox(
    neighborhood_map,
    lat="Lat",
    lon="Lon",
    size="gross_rent",
    zoom=10,
    color="neighborhood")

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [68]:
row1=pn.Row(avg_units, avg_rent_plot, avg_sales_plot)

row2=pn.Row(exp_neighborhoods, par_coor, par_categories)
row3=pn.Row(map_1, map_2)

In [69]:
SFO_dashboard=pn.Tabs(
    ("Average Housing Data", row1),
    ("Most Expensive Neighborhoods", row2),
    ("Neighborhood Maps", row3))

In [70]:
SFO_dashboard

Tabs
    [0] Row
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [1] Row
        [0] HoloViews(Bars)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)

## Serve the Panel Dashboard

In [71]:
SFO_dashboard.servable()

Tabs
    [0] Row
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [1] Row
        [0] HoloViews(Bars)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)